In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%load_ext autoreload
%autoreload 2

import os
ipy = get_ipython()
#data_dir_general = '/home/demitau/data_Quentin'
#data_subdir_mem_err_main = 'full_experiments/data2'
from os.path import join as pjoin
import os,sys; sys.path.append(os.path.expandvars('$CODE_MEMORY_ERRORS'))
import error_sensitivity
#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)
import pandas as pd

#data_dir_general = '/home/demitau/data_Quentin'
#data_subdir_mem_err_main = 'full_experiments/data2'
#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)
#scripts_dir = pjoin(data_dir_general,'full_experiments','scripts2')

data_dir_general = os.path.expandvars('$DATA_QUENTIN')
data_dir_input = os.path.expandvars('$DATA_MEMORY_ERRORS_STAB_AND_STOCH')
#scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'), 'previous_analyses')
scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'))


print(data_dir_input, scripts_dir)

subjects = [f for f in os.listdir(data_dir_input) if f.startswith('sub') ]
subjects = list(sorted(subjects))
print(subjects)

# Do manually

In [ ]:
from config2 import path_data
subject = subjects[0]

folder = pjoin(path_data, subject, 'behavdata')
files = os.listdir(folder)
fname_behavior = list()
task = 'visuomotor'
fname_behavior.extend([pjoin(folder, f) for f in files if ((task in f) and
                                                             ('.log' in f))])
print(fname_behavior)
fnf = fname_behavior[0]
# time is time since start
logcols = ('trials,trigger,target_inds,perturbation,jax1,jax2,'
    'feedbackX_screen,feedbackY_screen,'
    'org_feedbackX_screen,org_feedbackY_screen,'
    'error_distance,environment,time').split(',')
df = pd.read_csv(fnf, names=logcols)
assert (df['time'].diff().iloc[1:] > 0).all()

import numpy as np
target_angs = (np.array([157.5, 112.5, 67.5, 22.5]) + 90) * \
              (np.pi/180)

from config2 import stage2evn2event_ids,envcode2env
from base2 import width,height
from behav_proc import aggRows
df['subject'] = subject

In [ ]:
#len(logcols)

In [ ]:
stage2evn2event_ids

In [ ]:
phase2trigger = {
    'REST_PHASE': 10,
    'TARGET_PHASE': 20,
    'FEEDBACK_PHASE': 30,
    'ITI_PHASE': 40,
    'BREAK_PHASE': 50
}
trigger2phase = {
    10: 'REST_PHASE',
    20: 'TARGET_PHASE',
    30: 'FEEDBACK_PHASE',
    40: 'ITI_PHASE',
    50: 'BREAK_PHASE'
}

df['phase'] = df['trigger'].apply(lambda x: trigger2phase[x])

d = {'TARGET_PHASE':'target', 'FEEDBACK_PHASE':'feedback'}
df['env'] = df['environment'].apply(lambda x: envcode2env[x])

df['target_locs'] = df['target_inds'].apply(lambda x: target_angs[x])
df['target_locs'] = df['target_locs'] - np.pi
def f(x):
    ph = x['phase']
    if ph not in d:
        return -10000
    else:
        #print(ph)
        inds = stage2evn2event_ids[d[ph] ][x['env'] ]
        #print(inds, x['tgti'])
        if len(inds) == 1:
            r = inds[0]
        else:
            r = inds[x['target_inds']] 
        return r
    
df['target_codes'] = df.apply(f, 1)

df['org_feedbackX'] = df['org_feedbackX_screen'] - width /2 
df['org_feedbackY'] = df['org_feedbackY_screen'] - height /2 

df['feedbackX'] = df['feedbackX_screen'] - width /2 
df['feedbackY'] = df['feedbackY_screen'] - height /2 

# df = df.drop(columns=['feedbackX_screen','feedbackY_screen',
#                       'org_feedbackX_screen','org_feedbackY_screen'] )

#########

dfc_ = df.query('phase == "BREAK_PHASE"')
grp_ = dfc_.groupby(['trials'])
dfcpc = aggRows(dfc_ , 
    'time', 'max', grp_, coltake='corresp').copy()
dfcpc['perturbation'] = np.nan
dfcpc[['jax1','jax2','org_feedbackX','org_feedbackY',
      'feedbackX','feedbackY']] = np.nan

#########

#df['feedbackX'].isna().any()

from base2 import calc_rad_angle_from_coordinates
from base2 import radius
c = df['feedbackX'].isna()
df.loc[~c, 'feedback'] = calc_rad_angle_from_coordinates(
    df.loc[~c, 'feedbackX'].values, 
    df.loc[~c, 'feedbackY'].values, radius)
df['feedback'] = df['feedback'] - np.pi

c = df['org_feedbackX'].isna()
df.loc[~c, 'org_feedback'] = calc_rad_angle_from_coordinates(
    df.loc[~c, 'org_feedbackX'].values, 
    df.loc[~c, 'org_feedbackY'].values, radius)
df['org_feedback'] = df['org_feedback'] - np.pi

df['error'] = df['feedback'] - df['target_locs']
df['belief'] = df['org_feedback'] - df['target_locs']
dfc = df


grp = dfc.query('phase == "FEEDBACK_PHASE"').groupby(['trials'])
dfcc = aggRows(dfc, 'time', 'max', grp, coltake = 'corresp')
dfcc['time_diff'] = dfcc['time'].diff()

inds = dfcc.query('environment == 1').index
dfcc.loc[inds,'perturbation'] = dfcc.loc[inds,'feedback'] - dfcc.loc[inds,'org_feedback']


In [ ]:
df['org_feedbackX_screen']

In [ ]:
df['traj_dist_from_center'] = np.sqrt((df['jax1'] -  width /2)**2 +\
                                         (df['jax2'] -  height /2)**2)

In [ ]:
df['traj_dist_from_center']

In [ ]:
df.query('phase == "TARGET_PHASE" and trials == 0').\
    plot(x='time',y='traj_dist_from_center')

In [ ]:
df.query('phase == "TARGET_PHASE" and trials == 20')\
    [['trials','org_feedbackX','org_feedbackY','org_feedbackX_screen',
      'org_feedbackY_screen','org_dist_from_center']]

In [ ]:
from base2 import radius_cursor, radius_target
home_radius = radius_cursor + radius_target; home_radius

In [ ]:
# def f(df_):
#     return df_
# df.groupby

def f(df__):
    RT = np.nan
    #print()
    df_ = df.loc[df__.index]
    
    
    subdf0 = df_.query('phase == "TARGET_PHASE"')
    st_time = subdf0.iloc[0]['time']
    subdf1 = subdf0.query('traj_dist_from_center > @home_radius')
    exit_home_time = subdf1.iloc[0]['time']
    
    RT = exit_home_time - st_time
    #if len(subdf ):
    #    RT = subdf.iloc[0]['time'] - st_time
    #print(dur)
    #pd.DataFrame({'trials'})
    newdf = df_[['trials','time']].copy()
    newdf['RT'] = RT

    print(RT)
    raise ValueError('e')
    return newdf
assert df['time'].diff().min() > 0
r = df.groupby(['trials'], group_keys=False).apply(f)
df['RT'] = r['RT']

df[['trials','RT']].iloc[0]

In [ ]:
dfcc1R[['trials','RT']]

In [ ]:
df['RT'].describe()

In [ ]:
from behav_proc import addNonHitCol, addBehavCols
#dfos = addBehavCols(dfcc)

In [ ]:
#dfcc.plot(y='error', x='trials')

In [ ]:
# in romain data BREAK is not a separate trial,
# it is just a phase after condition change, around 25-33 sec

In [ ]:
FPS = 120
break_durations = dfc.query('phase == "BREAK_PHASE"').groupby('trials').size() / FPS
break_durations = break_durations.to_frame()
display(break_durations)

In [ ]:
print( 'timd_diff of dfcc ')
display(dfcc['time_diff'].describe()) 

In [ ]:
#grp = dfc.groupby(['trials','phase'])

In [ ]:
#.describe()
grp = dfc.groupby(['trials','phase'])
dfcc0 = aggRows(dfc, 'time', 'max', grp, coltake = 'corresp')
dfcc0['time_diff_phase'] = dfcc0['time'].diff()
dfcc0 = dfcc0.set_index(['trials','phase'])
dfcc0['phase_duration'] = aggRows(dfc, 'time', 'max', grp, coltake = 'corresp').set_index(['trials','phase'])['time'] -\
      aggRows(dfc, 'time', 'min', grp, coltake = 'corresp').set_index(['trials','phase'])['time']

dfcc0 = dfcc0.reset_index()
dfcc0 = dfcc0.sort_values('time')
dfcc0['time_since_last'] = \
    dfcc0.sort_values('time').groupby('phase')['time'].diff()

dfcc0['phase_duration'].describe()

In [ ]:
# from break Romain goes to ITI
# then trial index chanegx only after switch from ITI to REST

In [ ]:
#dfcc.query('time_diff > 10') 

In [ ]:
#dfcc0['trial_has_break'] = dfcc0['phase'] == 'BREAK_PHASE'

In [ ]:
trbr = dfcpc['trials'].unique()
dfcc0.sort_values('time').query('trials.isin(@trbr)')[['trials','phase','time']]

In [ ]:
#dfc.query('trials.isin(@trbr)').iloc[:800:20][['trials','phase','time']]

In [ ]:
# dfcc1 = dfcc.set_index('trials') 
# dfcc1['trial_has_break'] = dfcc0.groupby('trials').apply(
#     lambda dftmp: dftmp['trial_has_break'].any())

# assert (dfcc1['time'].diff().iloc[1:] > 0).all()

In [ ]:
dfcc0

In [ ]:
# definte break duration for trials where it happened
dfcc0['pre_break_duration'] = np.nan
c = dfcc0['phase'].isin( ["BREAK_PHASE"])
trials_with_break = dfcc0[c]['trials'].unique()
inds_ = dfcc0.query('trials.isin(@trials_with_break)').index
#dfcc0.loc[inds,'pre_break_duration'] = 

def f(df):
    dur = 0.
    #print()
    df_ = dfcc0.loc[df.index]
    subdf = df_.query('phase == "BREAK_PHASE"')
    if len(subdf ):
        dur = subdf.iloc[0]['phase_duration']
    #print(dur)
    #pd.DataFrame({'trials'})
    newdf = df_[['trials','phase']].copy()
    newdf['pre_break_duration'] = dur
    if dur > 1e-10:
        newdf['was_pre_break'] = True
    else:
        newdf['was_pre_break'] = False
    return newdf
r = dfcc0.groupby(['trials'], group_keys=False).apply(f)

dfcc0['pre_break_duration'] = r['pre_break_duration']
dfcc0['was_pre_break'] = r['was_pre_break']

I have phases, I need to set for each trial (or even for reach phase) a var of pre_break_dur to something

In [ ]:
# take only one phase
dfcc0_ = dfcc0.sort_values('time')
c = dfcc0_['phase'].isin( ["FEEDBACK_PHASE"])
dfcc1 = dfcc0_[c].copy()
print( len(dfcc), len(dfcc1) )
assert np.min(dfcc1['time'].diff().iloc[1:]) > 0
assert not dfcc1.duplicated(['trials']).any()

c = dfcc1['perturbation'].abs() > 1e-10
dfcc1.loc[c,'trial_type'] = 'perturbation'
dfcc1.loc[~c,'trial_type'] = 'veridical'
dfcc1.loc[dfcc1['phase'] == "BREAK_PHASE",'trial_type'] = 'pause'

###############  add more stuff from dfcc0
#dfcc2 = dfcc2.set_index(['trials','phase'])
qs0 = 'phase == "FEEDBACK_PHASE"'
dfcc1 = dfcc1.reset_index().sort_values('trials')
inds = dfcc1.query(qs0).index
dfcc0_ = dfcc0.reset_index().sort_values('trials')
dfcc1['home_duration'] = np.nan
dfcc1['movement_duration'] = np.nan
dfcc1['trial_duration'] = np.nan
qs = 'phase == "REST_PHASE"'
dfcc1.loc[inds,'home_duration']     = dfcc0_.query(qs)['phase_duration'].values
qs = 'phase == "TARGET_PHASE"'
dfcc1.loc[inds,'movement_duration'] = dfcc0_.query(qs)['phase_duration'].values

# not counting break part even if present
df_ = dfc.query('phase != "BREAK_PHASE"').groupby('trials')
trial_dur_excbreak = df_['time'].max() - df_['time'].min()
assert trial_dur_excbreak.to_frame().reset_index()['trials'].diff()[1:].min() > 0
dfcc1.loc[inds, 'trial_duration'] = trial_dur_excbreak.values

# if I want to compute time distance between target phase starts
# then I'd take dfcc0.query('TARGET_PHASE')['time_since_last']
dfcc1 = dfcc1.sort_values('time')

In [ ]:
# creating a "virtual trial" for break part
dfcc0_ = dfcc0.sort_values('time')
c = dfcc0_['phase'].isin( ["FEEDBACK_PHASE","BREAK_PHASE"])
dfcc0_['trial_index_inc_pause'] = c.cumsum() - 1

dfcc2 = dfcc0_[c].copy()
print( len(dfcc), len(dfcc2) )
assert np.min(dfcc2['time'].diff().iloc[1:]) > 0
assert not dfcc2.duplicated(['trial_index_inc_pause']).any()

c = dfcc2['perturbation'].abs() > 1e-10
dfcc2.loc[c,'trial_type'] = 'perturbation'
dfcc2.loc[~c,'trial_type'] = 'veridical'
dfcc2.loc[dfcc2['phase'] == "BREAK_PHASE",'trial_type'] = 'pause'

###############  add more stuff from dfcc0
#dfcc2 = dfcc2.set_index(['trials','phase'])
qs0 = 'phase == "FEEDBACK_PHASE"'
dfcc2 = dfcc2.reset_index().sort_values('trials')
inds = dfcc2.query(qs0).index
dfcc0_ = dfcc0.reset_index().sort_values('trials')
dfcc2['home_duration'] = np.nan
dfcc2['movement_duration'] = np.nan
dfcc2['trial_duration'] = np.nan
qs = 'phase == "REST_PHASE"'
dfcc2.loc[inds,'home_duration']     = dfcc0_.query(qs)['phase_duration'].values
qs = 'phase == "TARGET_PHASE"'
dfcc2.loc[inds,'movement_duration'] = dfcc0_.query(qs)['phase_duration'].values

# not counting break part even if present
df_ = dfc.query('phase != "BREAK_PHASE"').groupby('trials')
trial_dur_excbreak = df_['time'].max() - df_['time'].min()
assert trial_dur_excbreak.to_frame().reset_index()['trials'].diff()[1:].min() > 0
dfcc2.loc[inds, 'trial_duration'] = trial_dur_excbreak.values

# if I want to compute time distance between target phase starts
# then I'd take dfcc0.query('TARGET_PHASE')['time_since_last']

qs0 = 'phase == "BREAK_PHASE"'
inds = dfcc2.query(qs0).index.values
qs = 'phase == "BREAK_PHASE"'
dfcc2.loc[inds,'trial_duration'] = dfcc0_.query(qs)['phase_duration'].values

dfcc2 = dfcc2.sort_values('time')

'RT' when exit home_radius
'err'

In [ ]:
cols = ['trial_index_inc_pause','trials','phase','home_duration',
        'movement_duration','trial_duration','time_since_last']
dfcc2[cols]

In [ ]:
cols = ['trials','phase','home_duration',
        'movement_duration','trial_duration','time_since_last','pre_break_duration']
dfcc1[cols]

In [ ]:
dfcc1.query('was_pre_break == 1')

In [ ]:
dfcc1.columns

In [ ]:
dfcc1['movement_duration']#.isna().all()

In [ ]:
set(['trials', 'perturbation', 'error', 'err', 'trajectoryX', 'trajectoryY', 'feedbackX', 'feedbackY', 'feedback', 'org_feedback', 'belief', 'target_inds', 'target_codes', 'target_locs', 'RT', 'trial_duration', 'movement_duration', 'environment'])\
     - set(dfcc1.columns) 


In [ ]:
# dfcc2['home_duration'],
# dfcc2['movement_duration'],

In [ ]:
#dfcc2.query('trial_type == "pause"')

In [ ]:
subj = subjects[0]
behav_data_dir = pjoin(data_dir_input,'behavdata')
#behavdata

# Plot

In [ ]:
dfcc.plot(x='time',y='time_diff')
dfcc0.plot(x='time',y='time_diff_phase')

dfcc0.plot(x='time', y='time_diff_phase')

dfcc0.query('phase.isin(["FEEDBACK_PHASE","BREAK_PHASE"])')\
    .plot(x='time', y='trigger')

dfcc0.query('phase != "BREAK_PHASE"').plot(y='time_since_last')

# Run script

## quick test ES calc

In [ ]:
# recalc all behav essentially
#%debug
script_name = pjoin(scripts_dir,'preprocess_behav_calc_ES.py')
ipy.run_line_magic('run', f'-i {script_name} '
   '--use_sub_angles 1  --read_behav_version v2 --n_subjects 1 '
   '--do_read 0 --perturbation_random_recalc 1 --save_suffix "" '
  ' --trial_shift_size_max 2 --do_per_tgt 1 --do_per_env 1 --do_plot 0 --do_save 0'
    ' --retention_factor 1 --long_shift_numerator 0')
    
df_1 = df_all_multi_tsz
del df_all_multi_tsz

ipy.run_line_magic('run', f'-i {script_name} '
   '--use_sub_angles 1  --read_behav_version v2 --n_subjects 1 '
   '--do_read 0 --perturbation_random_recalc 1 --save_suffix "" '
  ' --trial_shift_size_max 2 --do_per_tgt 1 --do_per_env 1 --do_plot 0 --do_save 0'
    ' --retention_factor 1 --long_shift_numerator 1')
    
df_2 = df_all_multi_tsz

In [ ]:
qs = 'trial_shift_size == 2'
df1,df2  = df_1.query(qs),df_2.query(qs)

In [ ]:
-0.0284+0.0412, -0.0284+0.1015

In [ ]:
cols = ['subject_ind','block_name','trial_index','non_hit_not_adj','vals_for_corr1','vals_for_corr2','error','prevlike_error','correction','err_sens']
N = 4
display(df1[cols].iloc[:N])
display(df2[cols].iloc[:N])

qs = 'trial_shift_size == 1'
df1_,df2_  = df_1.query(qs),df_2.query(qs)
display(df1_[cols].iloc[:N])
display(df2_[cols].iloc[:N])


In [ ]:
df1['err_sens']

In [ ]:
df1['err_sens']

## Fast read_behav2_upd

In [ ]:
#for subject in subjects:
for subject in subjects[:1]:
    script_name = pjoin(scripts_dir,'read_behav2_upd.py')
    ipy.run_line_magic('run', f'-i {script_name} --subject {subject} --save_suffix ""')
    

In [ ]:
df.query('trials == 0')[['time','trigger','phase']]

In [ ]:
from behav_proc import checkErrBounds
checkErrBounds(dfcc1, ['feedback','org_feedback','belief'])

In [ ]:
dfcc1upd['error']

In [ ]:
from base2 import subAngles

In [ ]:
# import cmath
# from cmath import exp as cexp
# def subAngles(ang1, ang2):
#     if isinstance(ang1, pd.Series):
#         ang1 = ang1.values
#     if isinstance(ang2, pd.Series):
#         ang2 = ang2.values
#     r = np.exp(ang1 * 1j) * np.exp(-ang2 * 1j)
#     return np.log(r).imag

a1,a2 = 10,210
a1 = a1/  180 * np.pi
a2 = a2/  180 * np.pi
r = subAngles(a1,a2) / np.pi * 180
print(r)

subAngles(dfcc1upd['target_locs'], dfcc1upd['org_feedback'])

In [ ]:
import seaborn as sns
sns.lineplot(data=dfcc1, x='trials', y='error')

In [ ]:
#os.listdir('/home/demitau/data_Quentin/full_experiments/data2/sub01_WGPOZPEE/behavdata/')

## compare side by side

In [ ]:
subject = subjects[4]

script_name = pjoin(scripts_dir,'read_behav2.py')
ipy.run_line_magic('run', f'-i {script_name} --subject {subject} '
                   ' --save_suffix _test ')

script_name = pjoin(scripts_dir,'read_behav2_upd.py')
ipy.run_line_magic('run', f'-i {script_name} --subject {subject} '
                   ' --save_suffix _test  --use_sub_angles 1 '
                  ' --perturbation_random_recalc 0')

In [ ]:
df_ = df.query('environment == 1')
(df_['feedback'] - df_['org_feedback']).abs().max()

In [ ]:
df_ = dfcc.query('environment == 1')
(df_['perturbation']).abs().max()

In [ ]:
df_ = dfcc1.query('environment == 1')
(df_['perturbation']).abs().max()

In [ ]:
import datetime
#subject = 'sub04_XNDMUSRS'
fnf = f'/home/demitau/data_Quentin/full_experiments/data2/{subject}/behavdata/behav_VisuoMotor_df_upd_test.pkl'
dfcc1upd = pd.read_pickle(fnf)
print( str(datetime.datetime.fromtimestamp(os.path.getmtime(fnf))) )
fnf = f'/home/demitau/data_Quentin/full_experiments/data2/{subject}/behavdata/behav_VisuoMotor_dfv1_test.pkl'
dfcc1R = pd.read_pickle(fnf)
#from datetime import datetime
print(datetime.datetime.fromtimestamp(os.path.getmtime(fnf)))

for col in dfcc1R.columns:
    if col in dfcc1upd.columns:
        if dfcc1upd[col].dtype == 'O':
            print(col)
            continue
        dfd = (dfcc1upd[col] - dfcc1R[col]).abs().describe()
        if dfd['max'] > 1e-5:
            display(dfd)
        else:
            print(col,' consistent')

In [ ]:
checkErrBounds(dfcc1R), 

checkErrBounds(dfcc1upd)

checkErrBounds(dfcc1old)

In [ ]:
subject = subjects[4]
fnf = '/home/demitau/data_Quentin/full_experiments/data2/df_all_3.09.23.pkl.zip'
dfcc1old = pd.read_pickle(fnf)
adjust_PI = False
dfcc1old_ = dfcc1old.query('subject == @subject').copy()

In [ ]:
for col in dfcc1upd.columns:
    if col in dfcc1old_.columns:
        if dfcc1old_[col].dtype == 'O':
            print(col)
            continue
        dfd = (dfcc1old_[col] - dfcc1upd[col]).abs().describe()
        if dfd['max'] > 1e-5:
            display(dfd)
        else:
            print(col,' consistent')
       

In [ ]:
4.387671 - 2*np.pi

In [ ]:
dfcc1old_ = dfcc1old_.copy()
adjust_PI = True
adjustErrBoundsPi(dfcc1old_, ['feedback','org_feedback'])

In [ ]:
display( dfcc1old_.loc[:10,['trials','feedback']] )
display( dfcc1upd.loc[:10,['trials','feedback']] )
# - dfcc1R[col]

computeErrSens3 seems to give wrong sign of ES currently.
Belief_ in computeErrSens3 is computed without regard for constant shift in target_locs (and it's absense in org_feedback) though it does not seem to cause obvious problems


--use_sub_angles 0 gives consistency between (current version of) read_behav2 and read_behav2_up

September df_all is more consistent with use_sub_angles 1, except for feedback and org_feedback. But I apply adjustErrBoundsPi(dfcc1old_, ['feedback','org_feedback']), then it becomes consistent

In [ ]:
dfcc1old = dfcc1old.reset_index()

from behav_proc import addBehavCols, computeErrSensVersions, truncateDf
addBehavCols(dfcc1old)

df_all_multi_tsz_old, ndf2vn = computeErrSensVersions(dfcc1old, envs_cur,
    block_names_cur,pertvals_cur,gseqcs_cur,tgt_inds_cur,
    dists_rad_from_prevtgt_cur,dists_trial_from_prevtgt_cur,
    coln_nh = 'non_hit_not_adj',
    coln_nh_out = 'non_hit_shifted',
    computation_ver='computeErrSens3',
    subj_list = subjects, error_type=error_type,
    trial_shift_sizes = [1],
             addvars=[], use_sub_angles = use_sub_angles)

df_all_multi_tsz_old['err_sens']  = -df_all_multi_tsz_old['err_sens']

df_ = truncateDf(df_all_multi_tsz_old, q=0,infnan_handling='discard',coln='err_sens' )
#dfext = df_
me = df_.groupby(['subject','environment'], observed=True).\
    mean(numeric_only=1).reset_index()
#me.groupby(['env','thr']).size()
sns.set(font_scale=1.3)

from figure.mystatann import *

In [ ]:
fg = sns.catplot(data = me, kind='violin', y='err_sens', 
    x='environment',  order=[0,1], 
                 palette = ['tab:orange','tab:grey',], legend=None,
                bw = 0.5)
#addTitleInfo(fg.axes.flatten()[0])
for ax in fg.axes.flatten():
    ax.axhline(y=0, c='r', ls=':'); #ax.set_ylim(-5,5)
    
plotSigAll(ax, 0.8, 0.05, ticklen=0.02,
           df=me, coln='err_sens', colpair = 'environment')
if adjust_PI:
    plt.suptitle('Done djustment of org_feedback')

In [ ]:
df_all_upd = pd.read_pickle(
    '/home/demitau/data_Quentin/full_experiments/data2/df_all_test.pkl.zip')
df_all_upd = df_all_upd.reset_index()
from behav_proc import addBehavCols, computeErrSensVersions, truncateDf
addBehavCols(df_all_upd)

df_all_multi_tsz_upd, ndf2vn = computeErrSensVersions(df_all_upd, envs_cur,
    block_names_cur,pertvals_cur,gseqcs_cur,tgt_inds_cur,
    dists_rad_from_prevtgt_cur,dists_trial_from_prevtgt_cur,
    coln_nh = 'non_hit_not_adj',
    coln_nh_out = 'non_hit_shifted',
    computation_ver='computeErrSens3',
    subj_list = subjects, error_type=error_type,
    trial_shift_sizes = [1],
             addvars=[], use_sub_angles = use_sub_angles)


df_all_multi_tsz_upd['err_sens']  = -df_all_multi_tsz_upd['err_sens']

df_ = truncateDf(df_all_multi_tsz_upd, q=0,infnan_handling='discard',coln='err_sens' )
#dfext = df_
me = df_.groupby(['subject','environment'], observed=True).\
    mean(numeric_only=1).reset_index()
#me.groupby(['env','thr']).size()
sns.set(font_scale=1.3)

from figure.mystatann import *

fg = sns.catplot(data = me, kind='violin', y='err_sens', 
    x='environment',  order=[0,1], 
                 palette = ['tab:orange','tab:grey',], legend=None,
                bw = 0.5)
#addTitleInfo(fg.axes.flatten()[0])
for ax in fg.axes.flatten():
    ax.axhline(y=0, c='r', ls=':'); #ax.set_ylim(-5,5)
    
plotSigAll(ax, 0.8, 0.05, ticklen=0.02,
           df=me, coln='err_sens', colpair = 'environment')

In [ ]:
checkErrBounds(df_all_multi_tsz_upd)

In [ ]:
dfcc1R.query('environment == 1')['perturbation']

In [ ]:
dfcc1upd.query('environment == 1')['perturbation']

In [ ]:
4.319689 / np.pi * 180 - 90

In [ ]:
#x,y=0,1
x,y=-1,-1
calc_rad_angle_from_coordinates(np.array([x]),np.array([y]) ) / np.pi * 180

In [ ]:
import gc; gc.collect()

In [ ]:
dfcc1R.loc[304:306,'org_feedback']

In [ ]:
# df.query('phase.isin(["FEEDBACK_PHASE"])')\
#     [['trials','target_inds','target_codes','phase','env']]

In [ ]:
cols = ['phase','trials','perturbation','feedback']

In [ ]:
col = 'feedback'
dfcc1upd[np.abs(dfcc1upd[col] - dfcc1R[col]) > 0][cols]

In [ ]:
 dfcc1upd['target_locs_'] = dfcc1upd['target_locs'] + np.pi

In [ ]:
qs = ''
qs = 'environment == 0'
print('New')
display( dfcc1upd.query(qs)[cols] )
print('Romain')
display( dfcc1R.query(qs)[list(set(cols) & set(dfcc1R.columns))] )

In [ ]:
stage2evn2event_ids

In [ ]:
event_ids_tgt_random

In [ ]:
event_ids_tgt_stable

In [ ]:
set(dfcc1R.columns) - set(dfcc1upd.columns) 

In [ ]:
#for subject in subjects:
for subject in subjects[:1]:
    script_name = pjoin(scripts_dir,'read_behav2.py')
    ipy.run_line_magic('run', f'-i {script_name}')
    

## preproc_raw

In [ ]:
script_name = pjoin(scripts_dir,'preprocess_raw2.py')
ipy.run_line_magic('run', f'-i {script_name}')

In [ ]:
hpass = 'no_filter'

In [ ]:
raw = raw_backup.copy()

In [ ]:
raw_backup = raw.copy()

In [ ]:
if hpass == 'no_hpass':
    raw.filter(l_freq=None, h_freq=30, n_jobs=n_jobs)
elif hpass == 'no_hpass_tf':
    raw.filter(l_freq=None, h_freq=150, n_jobs=n_jobs)
elif hpass == 'no_filter':
    pass
else:
    # 30 -- just to smooth
    raw.filter(l_freq=float(hpass), h_freq=30, n_jobs=n_jobs)
if with_ICA:
    fname = op.join(path_data, subject, f'raw_{task}_{hpass}_with_ICA.fif')
else:
    fname = op.join(path_data, subject, f'raw_{task}_{hpass}.fif' )
raw.save(fname, overwrite=True)


In [ ]:
fname

In [ ]:
plt.hist( log[:,2] )

In [ ]:
targets

## long (all subjects)

In [ ]:
# todo edit error sensititivity calc: change numerator for shifts. Or maybe add a flag that changes it

In [ ]:
len(args.save_suffix)

In [ ]:
# recalc all behav essentially
#%debug
do_read = 0 # 1 means reprocess .csv
script_name = pjoin(scripts_dir,'preprocess_behav_calc_ES.py')
ipy.run_line_magic('run', f'-i {script_name} '
   '--use_sub_angles 1  --read_behav_version v2 '
   f'--do_read {do_read} --perturbation_random_recalc 1 --save_suffix "" '
  ' --trial_shift_size_max 4 --do_per_tgt 1 --do_per_env 1'
    ' --retention_factor 1.,0.924')

In [ ]:
radius_cursor, radius_target

In [ ]:
home_position = (400,400)

In [ ]:
pa`

In [ ]:
home

In [ ]:
params['ra']

In [ ]:
dfcc0.groupby(['phase'])['phase_duration'].describe()

In [ ]:
print(args.save_suffix)

In [ ]:
dftmp = df_all_multi_tsz.query('trial_shift_size == 1')
dftmp = truncateDf(dftmp, q=0,infnan_handling='discard',coln='err_sens' )    
me = dftmp.groupby(['subject','environment','trial_group_col_calc','retention_factor_s'], observed=True).\
    mean(numeric_only=1).reset_index() 

In [ ]:
me

In [ ]:
fg = sns.catplot(y='err_sens',x='environment',hue='environment',kind='violin',
            data=me, col='trial_group_col_calc',
            row='retention_factor_s') 

fg.refline(y=0)

In [ ]:
df_all_multi_tsz['retention_factor_s']

In [ ]:
df_ = truncateDf(df_, q=0,infnan_handling='discard',coln='err_sens' )

In [ ]:
df_all_multi_tsz.duplicated(['subject','trials','trial_group_col_calc','trial_shift_size','retention_factor_s']).any()

In [ ]:
args.retention_factor

In [ ]:
df_all_multi_tsz['trial_group_col_calc'].unique()

In [ ]:
df_all_multi_tsz.groupby(['subject','trials']).size().describe()

In [ ]:
import datetime
#datetime.datetime.fromtimestamp(os.path.getmtime(fname))
str( datetime.datetime.now() )[:10]

In [ ]:
script_name = pjoin(scripts_dir,'test_ES_calc.py')
ipy.run_line_magic('run', f'-i {script_name} '
                   '--use_sub_angles 1  --read_behav_version v2 '
                   '--do_read 0 --perturbation_random_recalc 1 --save_suffix _test')

In [ ]:
import pingouin as pg

In [ ]:
from behav_proc import comparePairs

In [ ]:
ttrssig, ttrs = comparePairs(df_, 'err_sens', 'environment')
assert ttrssig.query('ttstr == "0.0 > 1.0" and not pooled')['pval'].iloc[0] < 0.05

In [ ]:
pg.ttest(df_, 'environment')

In [ ]:
#order=[0,1], hue='environment',
#                 palette = ['tab:orange', 'tab:grey'], legend=None

In [ ]:
 df_all_multi_tsz['non_hit_not_adj'].shift()  

In [ ]:
#df_all_multi_tsz.loc[(~(df_all_multi_tsz['non_hit_not_adj'].shift(fill_value=True))), 'err_sens'] = np.nan
df_all_multi_tsz['err_sens'] = -df_all_multi_tsz['err_sens']

In [ ]:
df_ = df_all_multi_tsz.query('trial_shift_size == 1 and trial_group_col_calc == "trials"')
assert not df_.duplicated(['subject','trials']).any()

df_ = truncateDf(df_, q=0,infnan_handling='discard',coln='err_sens' )
#plt.figure()
me = df_.groupby(['subject','environment'], observed=True).\
    mean(numeric_only=1).reset_index()
#me.groupby(['env','thr']).size()
sns.set(font_scale=1.3)
fg = sns.catplot(data = me, kind='violin', y='err_sens', hue='environment',
    x='environment',palette = ['tab:orange', 'tab:grey'], legend=False)
#addTitleInfo(fg.axes.flatten()[0])
for ax in fg.axes.flatten():
    ax.axhline(y=0, c='r', ls=':'); #ax.set_ylim(-5,5)

from figure.mystatann import plotSigAll
plotSigAll(ax, 0.8, 0.05, ticklen=0.02,
       df=me, coln='err_sens', colpair = 'environment')


In [ ]:
script_name = pjoin(scripts_dir,'test_ES_calc.py')
ipy.run_line_magic('run', f'-i {script_name} '
                   '--use_sub_angles 0  --read_behav_version v1 '
                   '--do_read 1 --perturbation_random_recalc 1')

In [ ]:
#%debug
#addBehavCols(df_all)

In [ ]:
dfext = df_all.sort_values(['subject','trials']).copy()

In [ ]:
for subj in subjects:
    df_ = dfext.query('subject == @subj')
    inds = df_.index
    dfext.loc[inds,'_v'] = (dfext.loc[inds, 'target_locs'] - np.pi/2) - dfext.loc[inds, 'org_feedback']
    dfext.loc[inds,'correction'] = dfext.loc[inds,'_v'].diff()
    dfext.loc[inds,'err_sens']  = dfext.loc[inds,'correction'] / dfext.loc[inds,'error'].shift()

dfext['trial_shift_size'] = 1
dfext['trial_group_col_calc'] = 'trials'
dfext.loc[ ~dfext['non_hit_not_adj'].shift(fill_value = False),'err_sens'] = np.nan

In [ ]:
dfext['target_locs'].max()

In [ ]:
checkErrBounds(dfext, ['_v'])

In [ ]:
 (dfext['non_hit_not_adj'].shift())

In [ ]:
df_ = truncateDf(dfext, q=0,infnan_handling='discard',coln='err_sens' )
#dfext = df_
me = df_.groupby(['subject','environment'], observed=True).\
    mean(numeric_only=1).reset_index()
#me.groupby(['env','thr']).size()
sns.set(font_scale=1.3)
fg = sns.catplot(data = me, kind='violin', y='err_sens', 
    x='environment',  order=[0,1], hue = 'environment', palette = ['tab:orange','tab:grey',], legend=None)
#addTitleInfo(fg.axes.flatten()[0])
for ax in fg.axes.flatten():
    ax.axhline(y=0, c='r', ls=':'); #ax.set_ylim(-5,5)


In [ ]:
df_all_multi_tsz, ndf2vn = computeErrSensVersions(df_all, envs_cur,
    block_names_cur,pertvals_cur,gseqcs_cur,tgt_inds_cur,
    dists_rad_from_prevtgt_cur,dists_trial_from_prevtgt_cur,
    coln_nh = 'non_hit_not_adj',
    coln_nh_out = 'non_hit_shifted',
    computation_ver='computeErrSens3',
    subj_list = subjects, error_type=error_type,
    trial_shift_sizes = [1],
             addvars=[], use_sub_angles = use_sub_angles)


In [ ]:
## simple test

In [ ]:
def calcESnaive(dfext, subtract_pi2=1):
    for subj in dfext['subject'].unique():
        df_ = dfext.query('subject == @subj')
        inds = df_.index
        dfext.loc[inds,'_v'] = (dfext.loc[inds, 'target_locs']) - dfext.loc[inds, 'org_feedback']
        if subtract_pi2:
            dfext.loc[inds,'_v'] -= np.pi/2
        dfext.loc[inds,'correction'] = dfext.loc[inds,'_v'].diff()
        dfext.loc[inds,'err_sens']  = dfext.loc[inds,'correction'] / dfext.loc[inds,'error'].shift()
        dfext.loc[inds].loc[~dfext.loc[inds,'non_hit_not_adj'].\
                  shift(fill_value = False),'err_sens'] = np.nan

    dfext['trial_shift_size'] = 1
    dfext['trial_group_col_calc'] = 'trials'    
    cols = ['subject','trials','org_feedback','target_locs','error',
                   'non_hit_not_adj','_v','correction','err_sens']
    if 'perturbation' in dfext.columns:
        cols.append('perturbation')
    display(dfext[cols])

In [ ]:
dft = pd.DataFrame({'target_locs':[0,1.5], 'org_feedback':[0,1.25],
                   'perturbation':[0,0], 'noise':[0.1,0.1]})

dft['org_feedback'] = [0,1.45]
dft['trials'] = np.arange(len(dft))
# + or - perturbation?
dft['org_feedback'] += dft['noise']
dft['error'] = -dft['target_locs'] + ( dft['org_feedback'] + dft['perturbation'] )

dft['subject'] = 'test0'
dft['non_hit_not_adj'] = True
calcESnaive(dft, subtract_pi2=0)

In [ ]:
cols = ['subject','trials','org_feedback','target_locs','error',
        'non_hit_not_adj','perturbation']
sl = slice(22,27)
dft1 = df_all.iloc[sl][cols]
calcESnaive(dft1, subtract_pi2=0)

In [ ]:
dft2 = df_all_multi_tsz.iloc[sl][cols + ['belief_','belief','correction','err_sens']]
#calcESnaive(dft2)
display(dft2)

In [ ]:
checkErrBounds(df_all_multi_tsz,'belief_')

In [ ]:
df_all['mtime'].min()

In [ ]:
for subject in subjects:
#for subject in subjects[:1]:
    #script_name = pjoin(scripts_dir,'read_behav2.py')
    script_name = pjoin(scripts_dir,'read_behav2_upd.py')
    ipy.run_line_magic('run', f'-i {script_name}')
    
    del df
    del dfcc
    del dfcc1, dfcc2
    
    import gc; gc.collect()

In [ ]:
from behav_proc import *
#updstr = ''
updstr = '_upd'
behav_df_all = []
#or subj in subjects[:4]:#[si]
for subj in subjects:
    behav_data_dir = pjoin(data_dir_input,'behavdata')
    #behavdata
    task = 'VisuoMotor'
    fname = pjoin(path_data, subj, 'behavdata',
                    f'behav_{task}_df{updstr}.pkl' )
    # where we would save cleaned df
    #fname_EC = pjoin(path_data, subj, 'behavdata',
    #                f'behav_{task}_df_EC.pkl' )
    behav_df_full = pd.read_pickle(fname)

    behav_df = pd.read_pickle(fname)
    behav_df['subject'] = subj
    
    behav_df_all += [behav_df]
    
behav_df_all = pd.concat(behav_df_all)
# behav_df_all['correction'] = None
# behav_df_all['err_sens'] = None
df_all = behav_df_all.reset_index()

In [ ]:
import seaborn as sns
sns.lineplot(data=df_all, x='trials',y='error')
plt.figure()
sns.lineplot(data=df_all, x='trials',y='belief')
plt.figure()
sns.lineplot(data=df_all, x='trials',y='perturbation')

In [ ]:
df_all['perturbation'].max()

In [ ]:
# script_name = pjoin(scripts_dir,'figure','plot_behav2.py')
# ipy.run_line_magic('run', f'{script_name}')

## long test

In [ ]:
# recalc all behav essentially
#%debug
script_name = pjoin(scripts_dir,'test_ES_calc.py')
ipy.run_line_magic('run', f'-i {script_name} '
   '--use_sub_angles 1  --read_behav_version v2 '
   '--do_read 0 --perturbation_random_recalc 1 --save_suffix "" '
  ' --trial_shift_size_max 1 --do_per_tgt 0 --do_per_env 1 --do_save 0 '
    ' --coln_error error_pscadj --coln_correction_calc vals_for_corr_pscadj'
    ' --retention_factor 1.000') #0.924

## large ES

In [ ]:
s2bdf = {}
for subject in [subjects[0], subjects[1], subjects[2]]:
    script_name = pjoin(scripts_dir,'read_behav2.py')
    ipy.run_line_magic('run', f'-i {script_name} --subject {subject} '
                       ' --save_suffix _test ')

    s2bdf[subject] = behav_df

    

In [ ]:
ev = 0.131100
for subject,behav_df in s2bdf.items():
    print(subject)
    display( behav_df.query('abs(abs(error) - @ev ) < 1e-5')[cols0] )
    #display(behav_df.query('trials in @trs')[cols0])

In [ ]:
-0.063956 * 180 / np.pi

In [ ]:
display(behav_df.query('trials in @trs')[cols0])
behav_df.query('trials in @trs')['error']+ 2*np.pi

In [ ]:
# test
#%debug
script_name = pjoin(scripts_dir,'test_ES_calc.py')
s = (f'-i {script_name} '
   ' --read_behav_version v2 '
   '--do_read 0 --perturbation_random_recalc 1 --save_suffix ""'
  ' --trial_shift_size_max 1 --do_per_tgt 1 '
     '--do_per_env 1 --retention_factor 1.'
    ' --n_subjects 4 --do_save 0 --do_calc_ES 1 --do_plot 0')
ipy.run_line_magic('run', s + ' --reref_target_locs 0 --use_sub_angles 1 ')
df_all_multi_tsz0 = df_all_multi_tsz

ipy.run_line_magic('run', s + ' --reref_target_locs 1 --use_sub_angles 1 ')
df_all_multi_tsz1 = df_all_multi_tsz

ipy.run_line_magic('run', s + ' --reref_target_locs 0 --use_sub_angles 0 ')
df_all_multi_tsz2 = df_all_multi_tsz

ipy.run_line_magic('run', s + ' --reref_target_locs 1 --use_sub_angles 0 ')
df_all_multi_tsz3 = df_all_multi_tsz


In [ ]:
qs_sing = 'trial_shift_size == 1 and trial_group_col_calc == "trials" and retention_factor_s == "1.000"'
#qs = qs_sing 

In [ ]:
dftmp0 = df_all_multi_tsz0.query(qs_sing)
dftmp1 = df_all_multi_tsz1.query(qs_sing)
dftmp2 = df_all_multi_tsz2.query(qs_sing)
dftmp3 = df_all_multi_tsz3.query(qs_sing)

In [ ]:
m = ~np.isinf(dftmp0['err_sens'])
dfdif = (dftmp0.loc[m]['err_sens'] -\
     dftmp1.query(qs_sing).loc[m]['err_sens'] )

print( len(dfdif[dfdif > 1e-6]) )

In [ ]:
dftmp0.columns

In [ ]:
cols_ = ['subject_ind','trials','err_sens','correction',
         'vals_for_corr1','vals_for_corr2','org_feedback','target_locs',
         'home_duration','movement_duration']
qst = 'subject_ind == 4 and trials >= 305 and trials <= 308'
display( dftmp0.query(qst)[cols_] )
display( dftmp1.query(qst)[cols_] )
display( dftmp2.query(qst)[cols_] )
display( dftmp3.query(qst)[cols_] )

In [ ]:
from figure.plots import relplot_multi
r= relplot_multi(data=dftmp0.query(qst),
              ys=[['err_sens'],
                  ['correction'],                  
                  ['belief_'],                                    
                 ['target_locs'],
                 ['org_feedback'],
                 ], x='trials',
                height=1.7,aspect=4)
r[0].refline(y=0)
r[0].refline(x=307)

#['error'],
from figure.plots import relplot_multi
r= relplot_multi(data=dftmp1.query(qst),
              ys=[['err_sens'],
                  ['correction'],                  
                  ['belief_'],                                    
                 ], x='trials',
                height=1.7,aspect=4)
r[0].refline(y=0)
r[0].refline(x=307)

In [ ]:
12551	sub01_WGPOZPEE	455	-0.492990	8.679813	0.063956
67670	sub03_GPVDQMWB	278	0.298931	-9.314656	-0.063956

In [ ]:
badsubjs = ['sub01_WGPOZPEE','sub03_GPVDQMWB']
trials = [455,278]

In [ ]:
qs = 'subject.isin(@badsubjs) and trials.isin(@trials)'
dftmp = behav_df_all.query(qs)
dftmp = df_all_multi_tsz.query(qs +\
    ' and trial_shift_size == 1 and trial_group_col_calc == "trials" and retention_factor_s == "1.000"')
cols0 = ['subject','trials','error']
cols = cols0 + ['prev_error']
dftmp[cols]

In [ ]:
#ev = 0.041254
ev = 0.063956
trs = df_all.query('abs(abs(error) - @ev) < 1e-5')\
    ['trials'].values
df_all.query('abs(abs(error) - @ev) < 1e-5')[cols0]

In [ ]:
for rowi,row in behav_df_all.query('subject == @subjects[0]').iterrows():
    ev = row['error']
    ti = row['trials']
    dftmp = df_all.query('abs(abs(error) - @ev) < 1e-5')
    if len(dftmp) > 2:
        print(ti, f'{ev:.6f}', len(dftmp[cols0]), dftmp.subject.nunique() )

In [ ]:
pd.__version__

In [ ]:
dft = pd.DataFrame({'A':[0.1,0,2]})
dft['A'] = dft['A'].apply(lambda x:'f{x:.2f}')

# Questions
* mvt duration = cue (dissapearance of the cursor) till crossing the outer circle (reaching certain distance from the start)
* target_types, feeback_phase, target_phase -- ?
* ITI_duration = ?
* how large is noise?
* feedback vs org_feedback -- # feedback -- preceived feedback,  org_feedback -- feedback that would be perceived without perturbation